# Table Question Answering Pipeline

Experimenting with a TableQuestionAnsweringPipeline.

This is not really a good use of the model but it kind of works.

In [1]:
from transformers import pipeline

In [2]:
faker_mappings = {
    # Personal information
      "name": [
            "full name", "name", "name of user", "person name", "name of person", 
            "complete name", "user name", "customer name", "client name"
        ],
        "first_name": [
            "first name", "user's first name", "first name of a person", "person first name", 
            "given name", "forename", "christian name", '', ''
        ],
        "last_name": [
            "last name", "surname", "family name", "user's last name", 
            "last name of user", "person's surname", '', '', ''
        ],
        "password": [
            "user password", "person password", "member password", "secret password", 
            "confidential password", "example password", '', '', '' # all arrays have to be equal. 
        ]
}

In [3]:
pipe = pipeline(model="google/tapas-base-finetuned-wtq")

Device set to use mps:0


In [4]:
result = pipe(query=['user first name', 'full name', "secret password", "unique id"], table=faker_mappings)
print(result)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


[{'answer': 'first name', 'coordinates': [(0, 1)], 'cells': ['first name'], 'aggregator': 'NONE'}, {'answer': 'full name', 'coordinates': [(0, 0)], 'cells': ['full name'], 'aggregator': 'NONE'}, {'answer': 'secret password, confidential password', 'coordinates': [(3, 3), (4, 3)], 'cells': ['secret password', 'confidential password'], 'aggregator': 'NONE'}, {'answer': 'full name', 'coordinates': [(0, 0)], 'cells': ['full name'], 'aggregator': 'NONE'}]


In [5]:
# A function to lookup the faker function names based on coordinates
def lookup_keys_from_results(result, faker_mappings):
    reverse_mapping = {}
    for key, values in faker_mappings.items():
        for value in values:
            if value:
                reverse_mapping[value.lower()] = key
    
    coordinate_to_key = {}
    for item in result:
        answer = item['answer'].lower()
        
        if ',' in answer:
            answers = [a.strip() for a in answer.split(',')]
            for a in answers:
                if a in reverse_mapping:
                    for coord in item['coordinates']:
                        coordinate_to_key[coord] = reverse_mapping[a]
                    break
        else:
            if answer in reverse_mapping:
                for coord in item['coordinates']:
                    coordinate_to_key[coord] = reverse_mapping[answer]
    
    return coordinate_to_key

In [6]:
print(lookup_keys_from_results(result, faker_mappings))

{(0, 1): 'first_name', (0, 0): 'name', (3, 3): 'password', (4, 3): 'password'}
